# Train a logistic regression and gradient booster on 4 species

Friday, May 18, 2018

In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
import os
import sys

#Store the relative and absolute path to the home directory of the project
project_directory = '../..'
project_path = os.path.abspath(os.path.join(project_directory))

#Add the project path to the system path if necessesary, to be able to load modules from subdirectories
if project_path not in sys.path:
    sys.path.append(project_path)

In [4]:
import src.image_manager as im
import src.model_eval as meval

## Load the features dataframes and append one species to the 3 species

In [5]:
features_3species_path = os.path.join(project_path, 'data', 'features_3species_20180514.csv')
features_2species_path = os.path.join(project_path, 'data', 'features_alnus_thuja.csv')

In [7]:
acer_picea_platanus_df = im.load_df(features_3species_path)
alnus_thuja_df = im.load_df(features_2species_path)

In [8]:
acer_picea_platanus_df.head()

,p_hash,filename,folder,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,incv3_out_6,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
0,9bb759cb29b01a25,image_bigleaf_maple_57.png,acer_macrophyllum,0.180042,0.057599,0.632024,0.021612,0.297033,0.011966,0.903157,...,0.360490,0.009282,0.055258,0.110056,0.058888,0.201237,0.198077,1.394304,0.155102,0.000000
1,99c20b3b74d53de0,image_bigleaf_maple_43.png,acer_macrophyllum,0.667066,0.249269,0.074132,0.177577,0.520580,0.167225,0.233005,...,1.446164,0.920353,0.823285,0.617185,0.121126,0.065866,0.876259,0.226926,0.274808,0.000000
2,a1e1e0d4e646e765,image_big_leaf_maple_bark_48.png,acer_macrophyllum,0.159188,0.011783,0.187351,0.211521,0.466182,0.387081,0.217815,...,1.198391,0.460702,0.157247,0.267633,0.006349,0.466702,0.641492,1.066418,0.145887,0.223863
3,abee94c19cb28b0e,image_acer_macrophylum_tree_49.png,acer_macrophyllum,0.658278,0.010192,0.344359,0.192652,0.342696,0.362848,0.562928,...,0.257585,0.077631,0.057183,0.411228,0.417154,0.048589,0.421282,0.057896,0.325496,0.145896
4,d3c161631f2b4b4e,image_bigleaf_maple_94.png,acer_macrophyllum,0.056695,0.255299,0.407071,0.066481,0.000076,0.568202,0.362414,...,1.289151,0.578341,0.963826,0.028785,0.310883,0.001386,1.154405,0.034431,0.768076,0.217323


In [9]:
alnus_thuja_df.head()

,p_hash,filename,folder,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,incv3_out_6,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
1262,c5c3866ab4dc8637,image_western_red_cedar_tree_17.png,thuja_plicata,1.336279,0.011943,0.395570,0.218817,0.317935,0.056648,0.087848,...,0.433737,0.159791,0.323293,0.009519,0.028450,0.134239,0.136767,0.110591,0.925633,0.000594
1263,83955f6a27e8417c,image_western_red_cedar_27.png,thuja_plicata,0.043319,0.085838,0.346090,0.269240,0.104244,0.059988,0.322002,...,0.215635,0.038550,0.123750,0.001528,0.207110,0.146056,0.085234,0.053560,0.083258,0.380264
1264,d3d39896b086f45a,image_western_red_cedar_33.png,thuja_plicata,1.537653,0.012991,0.189000,0.226903,0.465502,0.050164,0.166596,...,0.508620,0.233280,0.289102,0.243015,0.178973,0.020320,0.253384,0.242000,0.159114,0.119313
1265,8d889a9666a9d99b,image_thuja_plicata_branches_58.png,thuja_plicata,0.777048,0.043122,0.167806,0.864698,0.690728,0.504618,0.055242,...,0.283768,0.297945,0.192668,0.231448,0.173066,0.011833,0.944232,0.390534,0.715745,0.000000
1266,9d00e21bd27c4fb5,image_thuja_plicata_cones_58.png,thuja_plicata,0.117833,0.227984,0.097914,0.286688,0.123003,0.693201,0.113581,...,0.804989,0.391352,0.548050,0.849486,0.280309,0.712661,0.956842,0.001279,0.351022,0.132506


In [16]:
features_4species_df = acer_picea_platanus_df.append(alnus_thuja_df[alnus_thuja_df['folder'] == 'thuja_plicata'])
features_4species_df.tail()

,p_hash,filename,folder,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,incv3_out_6,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
1832,e36898c66d399cb4,image_thuja_plicata_branches_96.png,thuja_plicata,0.083918,0.166168,0.558727,0.027537,0.054673,0.170166,0.166989,...,1.370445,0.303498,0.222401,0.019987,0.021358,0.235605,0.032119,0.538935,0.037923,1.102876
1833,a9e4d68bd7632870,image_thuja_plicata_cones_82.png,thuja_plicata,0.340928,0.050327,0.118958,0.220579,0.010450,0.091026,0.316154,...,0.338676,0.028837,0.201438,0.922495,0.369425,0.020133,0.139394,0.000000,0.111569,0.724462
1834,d79ae0359f48835a,image_thuja_plicata_branches_82.png,thuja_plicata,0.650036,0.399849,0.566335,0.157019,0.792458,0.424557,0.438763,...,0.641063,0.229398,0.694067,0.553304,0.000000,0.218311,1.626439,0.020453,0.323357,0.070934
1835,a6a6f57594f284d0,image_western_red_cedar_tree_32.png,thuja_plicata,0.258416,0.010987,0.227817,0.598694,0.346953,0.114962,0.037960,...,0.220482,0.334772,0.886903,0.891988,0.353683,0.047528,0.631834,0.000000,0.044669,0.544044
1836,abd14c341baf47c2,image_western_red_cedar_tree_26.png,thuja_plicata,0.193753,0.006266,0.632279,0.065293,0.332640,0.151659,0.476420,...,0.128943,0.060852,0.000000,0.025686,0.029323,0.328659,0.144968,0.081436,0.771464,0.158379


In [19]:
features_4species_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2214 entries, 0 to 1836
Columns: 2051 entries, p_hash to incv3_out_2047
dtypes: float64(2048), object(3)
memory usage: 34.7+ MB


In [17]:
len(alnus_thuja_df[alnus_thuja_df['folder'] == 'thuja_plicata'])

575

In [18]:
1639+575 #Check that the totals for the 4 species add up

2214

## Pull out features (X) and labels (y), and get a train-test split¶

In [20]:
X = features_4species_df.drop(['p_hash', 'filename', 'folder'], axis=1)
y = features_4species_df['folder']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=987)

In [23]:
print(f'Train: {len(X_train)}, Test: {len(X_test)}')

Train: 1771, Test: 443


In [24]:
y_train.value_counts()

picea_sitchensis       462
thuja_plicata          458
acer_macrophyllum      430
platanus_acerifolia    421
Name: folder, dtype: int64

In [25]:
y_test.value_counts()

platanus_acerifolia    120
thuja_plicata          117
picea_sitchensis       110
acer_macrophyllum       96
Name: folder, dtype: int64

In [29]:
np.log(4) #Baseline for log loss

1.3862943611198906

## Try Ridge Logistic Regression with same C as before

In [31]:
log_model = LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='sag', C=0.001, max_iter=4000)

In [32]:
log_model.fit(X_train, y_train)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False)

In [33]:
logeval = meval.ModelEvaluator(log_model)
logeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.8137722376015083, Test log_loss: 0.861858336866128
Train accuracy: 0.7594579333709769, Test accuracy: 0.6930022573363431


## Do a grid search to find the best C

In [35]:
param_grid = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1]}
gridsearch = GridSearchCV(estimator=log_model,
                          param_grid=param_grid, 
                          scoring=['neg_log_loss', 'accuracy'],
                          refit='neg_log_loss')

In [36]:
gridsearch.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit='neg_log_loss',
       return_train_score='warn', scoring=['neg_log_loss', 'accuracy'],
       verbose=0)

In [37]:
gridsearch.best_params_

{'C': 0.1}

In [38]:
gridsearch.best_estimator_

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False)

In [39]:
logeval = meval.ModelEvaluator(gridsearch.best_estimator_)
logeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.16499753162411077, Test log_loss: 0.6367373489170458
Train accuracy: 0.9881422924901185, Test accuracy: 0.7539503386004515


Whoa, nice! This beat my 3 species model... Hmm, maybe I should go back and see if I really found the best C for 3 species...

## Since the best C was at the edge of our range, let's do another search to see if we can do better

Figure out a logarithmic scale of parameters to use

In [42]:
10**(1/3)

2.154434690031884

In [44]:
[2.154434690031884**x for x in range(4)]

[1.0, 2.154434690031884, 4.641588833612779, 10.000000000000002]

In [49]:
np.sqrt(10)

3.1622776601683795

In [52]:
[3.16**x for x in range(5)]

[1.0, 3.16, 9.985600000000002, 31.554496000000004, 99.71220736000002]

In [56]:
[3**x for x in range(-2,3)]

[0.1111111111111111, 0.3333333333333333, 1, 3, 9]

In [57]:
param_grid = {'C': [0.03, 0.1, 0.3, 1, 10]}
gridsearch = GridSearchCV(estimator=log_model,
                          param_grid=param_grid, 
                          scoring=['neg_log_loss', 'accuracy'],
                          refit='neg_log_loss')

In [58]:
gridsearch.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.03, 0.1, 0.3, 1, 10]}, pre_dispatch='2*n_jobs',
       refit='neg_log_loss', return_train_score='warn',
       scoring=['neg_log_loss', 'accuracy'], verbose=0)

In [59]:
gridsearch.best_params_

{'C': 0.1}

Ok, so we got the same value of C

In [60]:
log_model

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False)

In [61]:
log_model = gridsearch.best_estimator_
log_model

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=4000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='sag', tol=0.0001, verbose=0,
          warm_start=False)

In [63]:
logeval.confusion_df(X_test, y_test)

,Predicted acer_macrophyllum,Predicted picea_sitchensis,Predicted platanus_acerifolia,Predicted thuja_plicata
Actual acer_macrophyllum,68,4,14,10
Actual picea_sitchensis,5,89,1,15
Actual platanus_acerifolia,27,2,87,4
Actual thuja_plicata,9,16,2,90


## Let's try Lasso Logistic Regression

In [64]:
llog_model = LogisticRegression(penalty='l1', multi_class='multinomial', solver='saga', C=0.1, max_iter=4000)

In [65]:
llog_model.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [66]:
llogeval = meval.ModelEvaluator(llog_model)
llogeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.5923158627730702, Test log_loss: 0.7438334342573347
Train accuracy: 0.8063241106719368, Test accuracy: 0.7065462753950339


In [67]:
param_grid = {'C': [0.01, 0.1, 1]}
gridsearch = GridSearchCV(estimator=llog_model,
                          param_grid=param_grid, 
                          scoring=['neg_log_loss', 'accuracy'],
                          refit='neg_log_loss')

In [68]:
gridsearch.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1]}, pre_dispatch='2*n_jobs',
       refit='neg_log_loss', return_train_score='warn',
       scoring=['neg_log_loss', 'accuracy'], verbose=0)

In [70]:
pd.DataFrame(gridsearch.cv_results_)

/Users/ndbs/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/ndbs/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_neg_log_loss'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/ndbs/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/ndbs/anaconda3/lib

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_neg_log_loss,mean_train_accuracy,mean_train_neg_log_loss,param_C,params,rank_test_accuracy,rank_test_neg_log_loss,...,split2_test_accuracy,split2_test_neg_log_loss,split2_train_accuracy,split2_train_neg_log_loss,std_fit_time,std_score_time,std_test_accuracy,std_test_neg_log_loss,std_train_accuracy,std_train_neg_log_loss
0,2.438513,0.006110,0.400339,-1.318793,0.399496,-1.316427,0.01,{'C': 0.01},3,3,...,0.419355,-1.321498,0.399323,-1.309506,0.261535,0.000128,0.013434,0.002502,0.005467,0.006403
1,22.105378,0.006597,0.706381,-0.760069,0.782036,-0.660905,0.1,{'C': 0.1},2,2,...,0.702886,-0.769943,0.789340,-0.644273,2.264667,0.000467,0.008929,0.007087,0.007943,0.012517
2,110.493191,0.006627,0.738566,-0.698227,0.994636,-0.125886,1,{'C': 1},1,1,...,0.723260,-0.736571,0.992386,-0.123835,11.152733,0.000974,0.015212,0.031465,0.002111,0.003890


In [71]:
gridsearch.best_params_

{'C': 1}

In [72]:
gridsearch.best_estimator_

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [73]:
llogeval = meval.ModelEvaluator(gridsearch.best_estimator_)
llogeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.11672207085325637, Test log_loss: 0.8140612610303455
Train accuracy: 0.9937888198757764, Test accuracy: 0.7404063205417607


In [76]:
llogeval = meval.ModelEvaluator(gridsearch.best_estimator_)
llogeval.confusion_df(X_test, y_test)

,Predicted acer_macrophyllum,Predicted picea_sitchensis,Predicted platanus_acerifolia,Predicted thuja_plicata
Actual acer_macrophyllum,68,5,13,10
Actual picea_sitchensis,6,88,3,13
Actual platanus_acerifolia,28,5,84,3
Actual thuja_plicata,9,17,3,88


### Let's try one more C value since our optimum was on the boundary of the grid

This took several minutes to train.

In [77]:
llog_model = LogisticRegression(penalty='l1', multi_class='multinomial', solver='saga', C=10, max_iter=4000)
llog_model.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [78]:
llogeval = meval.ModelEvaluator(llog_model)
llogeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.012917768707346471, Test log_loss: 1.2342111171041998
Train accuracy: 1.0, Test accuracy: 0.7404063205417607


Ok, so the results on the test set don't improve at all -- instead, it just overfits even more badly to the training data. So the best model is still with `C=1`. (Haha check out the confusion matrix for the training set!)

In [80]:
llogeval.confusion_df(X_train, y_train)

,Predicted acer_macrophyllum,Predicted picea_sitchensis,Predicted platanus_acerifolia,Predicted thuja_plicata
Actual acer_macrophyllum,430,0,0,0
Actual picea_sitchensis,0,462,0,0
Actual platanus_acerifolia,0,0,421,0
Actual thuja_plicata,0,0,0,458


In [81]:
llogeval.confusion_df(X_test, y_test)

,Predicted acer_macrophyllum,Predicted picea_sitchensis,Predicted platanus_acerifolia,Predicted thuja_plicata
Actual acer_macrophyllum,67,5,14,10
Actual picea_sitchensis,6,88,4,12
Actual platanus_acerifolia,27,5,83,5
Actual thuja_plicata,9,16,2,90


In [83]:
llog_model = gridsearch.best_estimator_
llog_model

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

## All right, let's try a gradient booster

In [84]:
gb_model = GradientBoostingClassifier(learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=5)

In [85]:
gb_model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=0.5, verbose=0,
              warm_start=False)

In [86]:
gbeval = meval.ModelEvaluator(gb_model)
gbeval.print_classifier_metrics(X_train, X_test, y_train, y_test)

Train log_loss: 0.38322303020846127, Test log_loss: 0.8374452604196712
Train accuracy: 0.9949181253529079, Test accuracy: 0.7155756207674944


In [92]:
gb_model = GradientBoostingClassifier(learning_rate=0.05, n_estimators=250, subsample=0.5,
                                      max_features='auto', max_depth=3, verbose=1)